# Wavespectra tutorial

## Links

### Github repository:
https://github.com/wavespectra/wavespectra

### Documentation:
https://wavespectra.readthedocs.io/en/latest/

## Installation

### Pre-install dependencies

* Linux, Windows and MacOS
* python3 (3.8)
* With pip:
    * Fortran compiler
    * Numpy
* With conda
    * None

### Installing with pip

To install with pip ensure a Fortran compiler such as gfortran is available and that numpy is pre-installed. Then run on a Terminal:

    pip install wavespectra

or alternatively:

    pip install wavespectra[extra]

for a complete installation including some core libraries such as netcdf and zarr.

### Installing with conda

Conda should take care of pre-install dependencies for you. Simply run:

    conda install -c conda-forge wavespectra

## Introduction

Wavespectra is an open source project for processing ocean wave spectral data. The library is built on top of xarray and focussed on speed and efficiency for large numbers of spectra.

### Xarray

[xarray](https://docs.xarray.dev/en/stable/) is a very useful python library designed to handle labelled multi-dimensional arrays in an intuitive way.

In [1]:
from pathlib import Path
import xarray as xr

datadir = Path("../data")

dset = xr.open_dataset(datadir / "ww3file.nc", engine="netcdf4")
dset

<xarray.Dataset>
Dimensions:    (direction: 24, time: 9, station: 2, frequency: 25)
Coordinates:
  * direction  (direction) float32 90.0 75.0 60.0 45.0 ... 135.0 120.0 105.0
  * frequency  (frequency) float32 0.04118 0.0453 0.04983 ... 0.3687 0.4056
  * station    (station) int32 1 2
  * time       (time) datetime64[ns] 2014-12-01 ... 2014-12-05
Data variables:
    dpt        (time, station) float32 ...
    efth       (time, station, frequency, direction) float32 ...
    latitude   (time, station) float32 ...
    longitude  (time, station) float32 ...
    wnd        (time, station) float32 ...
    wnddir     (time, station) float32 ...

One of the core objects in xarray is a `Dataset` which is composed by labelled attributes, dimensions, coordinates and data variables, resambling the netcdf data model. The data variables are `DataArray` objects which are multi-dimensional arrays that can share a set of dimensions and coordinates in the Dataset.

These labels can be used to define operations on the data in a very simple but powerful way. 

In [2]:
# Selecting single spectrum from the dataset

dset.sel(time="2014-12-01 00:00", station=1, drop=True)

<xarray.Dataset>
Dimensions:    (direction: 24, frequency: 25)
Coordinates:
  * direction  (direction) float32 90.0 75.0 60.0 45.0 ... 135.0 120.0 105.0
  * frequency  (frequency) float32 0.04118 0.0453 0.04983 ... 0.3687 0.4056
Data variables:
    dpt        float32 106.6
    efth       (frequency, direction) float32 3.206e-15 6.08e-15 ... 0.00148
    latitude   float32 19.95
    longitude  float32 92.1
    wnd        float32 5.1
    wnddir     float32 24.92

In [3]:
# Accessing single variable in the Dataset

dset.wnd

<xarray.DataArray 'wnd' (time: 9, station: 2)>
array([[5.099654, 5.478037],
       [6.14928 , 5.787188],
       [3.290406, 3.389233],
       [6.259294, 6.111243],
       [4.355951, 4.619362],
       [6.507405, 6.373147],
       [3.741731, 3.732023],
       [4.522705, 4.199544],
       [3.270287, 2.889581]], dtype=float32)
Coordinates:
  * station  (station) int32 1 2
  * time     (time) datetime64[ns] 2014-12-01 2014-12-01T12:00:00 ... 2014-12-05
Attributes:
    long_name:      wind speed at 10m
    standard_name:  wind_speed
    globwave_name:  wind_speed
    units:          m s-1
    valid_min:      0.0
    valid_max:      100.0
    content:        TX
    associates:     time station

In [4]:
# Applying operations across dimensions

dset.wnd.mean()

<xarray.DataArray 'wnd' ()>
array(4.765337, dtype=float32)

### Wavespectra

Wavespectra defines two main objects analogue to xarray's Dataset and DataArray:

`Dataset` --> `SpecDataset`

`DataArray` -- > `SpecArray`

These objects (and their bounded methods) are accessed from the **.spec** namespace defined by wavespectra:

In [5]:
from wavespectra import read_ww3

dset = read_ww3(datadir / "ww3file.nc")

dset.spec

<SpecDataset>
Dimensions:  (dir: 24, time: 9, site: 2, freq: 25)
Coordinates:
  * dir      (dir) float32 270.0 255.0 240.0 225.0 ... 330.0 315.0 300.0 285.0
  * freq     (freq) float32 0.04118 0.0453 0.04983 ... 0.3352 0.3687 0.4056
  * site     (site) int32 1 2
  * time     (time) datetime64[ns] 2014-12-01 2014-12-01T12:00:00 ... 2014-12-05
Data variables:
    dpt      (time, site) float32 dask.array<SpecDataset>
    efth     (time, site, freq, dir) float32 dask.array<SpecDataset>
    lat      (site) float32 dask.array<SpecDataset>
    lon      (site) float32 dask.array<SpecDataset>
    wspd     (time, site) float32 dask.array<SpecDataset>
    wdir     (time, site) float32 dask.array<SpecDataset>

### conventions

Wavespectra defines some conventions around the units and names for some variables and spectral dimensions:

* **freq** (Hz)
* **dir** (degree, coming_from)
* **efth** (m2s/degree)
* **wspd** (m/s)
* **wdir** (degree, coming_from)

### Interface

The spec namespace provides an interface for processing wave spectra data:

* Reading from and writing to several wave spectra formats
* Calcule spectral statistics
* Spectral partitioning
* Plotting
* Selecting from pointcloud datasets
* Spectra reconstruction (under development)